In [1]:
!pip install xgboost
!pip install flask_cors

In [2]:
! pip install --user graphistry

In [3]:
import pandas as pd
import numpy as np
import requests
from flask import Flask, request, json, jsonify
from flask_cors import CORS
import pickle
from concurrent.futures import ThreadPoolExecutor
import graphistry
from multiprocessing import Process
import asyncio

In [4]:
model = pickle.load(open('XGB_FRAUD1.pickle', 'rb'))

### 웹 연동

In [5]:
app = Flask(__name__)
CORS(app)

import requests
import time
user_adr='1CRCqjGKgiNziUi9k5aqtbcLp3575BRQKB'
def get_url(url):
    return requests.get(url)

list_of_urls =['https://blockchain.info/rawaddr/'+user_adr+'?limit=10','https://www.bitcoinabuse.com/api/reports/check?address='+user_adr+'&api_token=GWGXyrT3Yil1PF8hZ49EEP6X8LiOkSr4KzA9bIDc']

with ThreadPoolExecutor(max_workers=2) as pool:
    response_list = list(pool.map(get_url,list_of_urls))
    
for response in response_list:
    print(response)
    
    
report1 = get_url(list_of_urls[0])
report2 = get_url(list_of_urls[1])

search_json = report1.json()
report_json = report2.json()

report_url = 'https://www.bitcoinabuse.com/api/reports/check?address=1CRCqjGKgiNziUi9k5aqtbcLp3575BRQKB&api_token=GWGXyrT3Yil1PF8hZ49EEP6X8LiOkSr4KzA9bIDc'

report_adrlist = requests.request("GET", report_url)

report_adrlist.json()

In [ ]:
@app.route("/user_csv", methods=['POST'])
def user_csv():
    # 1. 웹으로부터 csv파일 받기
    csv_form = request.form.get('csv_form')
    user_csv = request.files['user_csv']
    print(user_csv)
    data = pd.read_csv(user_csv)
   
    # 2. 모델에 csv파일 넣어서 분석 -> 도넛그래프
    preds_xgb = model.predict(data)
    result_csvlist = preds_xgb.tolist()
    data['FLAG'] = preds_xgb
    
    # 3. api에 csv파일 넣어서 막대그래프
    rows=data.shape[0]

    sum1 = 0
    sum2 = 0
    sum3 = 0
    sum4 = 0
    sum5 = 0
    absum1 = 0
    absum2 = 0
    absum3 = 0
    absum4= 0
    absum5 = 0
    cnt = 0
    abcnt = 0

    for i in range(rows):
        if(data['FLAG'][i]==0):
            cnt= cnt+1
            sum1=sum1+data['total tnx'][i]
            sum2=sum2+data['Time Diff between first and last (Mins)'][i]
            sum3=sum3+data['total ether balance'][i]
            sum4=sum4+data['avg total val'][i]                                             
            sum5=sum5+data['max value received '][i]
        else:
            abcnt= abcnt+1
            absum1=absum1+data['total tnx'][i]
            absum2=absum2+data['Time Diff between first and last (Mins)'][i]
            absum3=absum3+data['total ether balance'][i]
            absum4=absum4+data['avg total val'][i]
            absum5=absum5+data['max value received '][i]
    col1_0 = sum1/cnt
    col1_1 = absum1/abcnt
    col2_0 = sum2/cnt/60
    col2_1 = absum2/abcnt/60
    col3_0 = sum3/cnt
    col3_1 = absum3/abcnt
    col4_0 = sum4/cnt
    col4_1 = absum4/abcnt
    col5_0 = sum5/cnt
    col5_1 = absum5/abcnt

    nomal_list=[col1_0,col2_0,col3_0,col4_0,col5_0]
    abnomal_list=[col1_1,col2_1,col3_1,col4_1,col5_1]
    
    # 4. api에 csv파일 넣어서 관계그래프
    graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="ddosby", password="tnsmd690@") 
    g = graphistry.edges(data).bind(source='max value received ', destination='FLAG')
    g_page = g.plot().data
    
    # 5. json타입으로 웹으로 전송
    return jsonify(result_csvlist, nomal_list, abnomal_list, g_page)

@app.route("/user_adr", methods=['GET'])
def user_adr():
    # 1. 웹으로부터 지갑주소 받기
    user_adr = request.args.get('user_adr', 'user_adr')
    
    # 2. api에 지갑주소 넣어서 조회
    search_url = 'https://blockchain.info/rawaddr/'+user_adr+'?limit=10'
    
    # 3. 결과데이터 리스트에 담기
    search_adrlist = requests.request("GET", search_url)
    search_adr_json = search_adrlist.json()
    
    # 4. json타입으로 웹으로 전송
    return search_adr_json

@app.route("/user_report", methods=['GET'])
def user_report():
    # 1. 웹으로부터 지갑주소 받기
    user_adr = request.args.get('user_adr', 'user_adr')
    
    # 2. api에 지갑주소 넣어서 조회
    report_url = 'https://www.bitcoinabuse.com/api/reports/check?address='+user_adr+'&api_token=GWGXyrT3Yil1PF8hZ49EEP6X8LiOkSr4KzA9bIDc'
    
    
    # 3. 결과데이터 리스트에 담기
    report_adrlist = requests.request("GET", report_url)
    report_adr_json = report_adrlist.json()
    
    # 4. json타입으로 웹으로 전송
    return report_adr_json

@app.route("/g_fds", methods=['GET'])
def g_fds():
 
    graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="ddosby", password="tnsmd690@")   
    hg = graphistry.hypergraph(pd.read_csv('./bnode.csv'), ['txId1', 'txId2'], direct=True)
    g= hg['graph']
    data=g.plot().data
    
    # 4. json타입으로 웹으로 전송
    return jsonify(data)
    

# 전송
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)

 127.0.0.1 - - [18/Dec/2022 17:35:40] "GET /user_report?user_adr=1PuHFFK1ZnxScywgDRcuWeyEWfXCqevk8o HTTP/1.1" 200 -

 127.0.0.1 - - [18/Dec/2022 23:56:11] "GET /user_adr?user_adr=1PuHFFK1ZnxScywgDRcuWeyEWfXCqevk8o HTTP/1.1" 200 -

 127.0.0.1 - - [18/Dec/2022 23:57:15] "GET /user_report?user_adr=1PuHFFK1ZnxScywgDRcuWeyEWfXCqevk8o HTTP/1.1" 200 -

 127.0.0.1 - - [19/Dec/2022 00:04:39] "GET /user_report?user_adr=width%3A%20200px%3B1PuHFFK1ZnxScywgDRcuWeyEWfXCqevk8o HTTP/1.1" 200 -

 127.0.0.1 - - [19/Dec/2022 00:04:44] "GET /user_report?user_adr=1PuHFFK1ZnxScywgDRcuWeyEWfXCqevk8o HTTP/1.1" 200 -

 127.0.0.1 - - [19/Dec/2022 00:05:46] "GET /user_report?user_adr=1PuHFFK1ZnxScywgDRcuWeyEWfXCqevk8o HTTP/1.1" 200 -

 127.0.0.1 - - [19/Dec/2022 00:10:35] "GET /user_report?user_adr=1PuHFFK1ZnxScywgDRcuWeyEWfXCqevk8o HTTP/1.1" 200 -



<FileStorage: 'data_01_test .csv' ('text/csv')>


 127.0.0.1 - - [19/Dec/2022 00:18:47] "POST /user_csv HTTP/1.1" 200 -



@app.route("/user_adr", methods=['GET'])
def user_adr():
    
    # 1. 웹으로부터 지갑주소 받기
    user_adr = request.args.get('user_adr', 'user_adr')
        
        
       
    # 2. api에 지갑주소 넣어서 조회
    async def search(user_adr):
            search_url = 'https://blockchain.info/rawaddr/'+user_adr+'?limit=10'
            search_adrlist = requests.request("GET", search_url)
            search_adr_json = search_adrlist.json()
            return search_adr_json
    async def report(user_adr):
            report_url = 'https://www.bitcoinabuse.com/api/reports/check?address='+user_adr+'&api_token=GWGXyrT3Yil1PF8hZ49EEP6X8LiOkSr4KzA9bIDc'
            report_adrlist = requests.request("GET", report_url)
            report_adr_json = report_adrlist.json()
            return report_adr_json
    async def main(user_adr):
         task1 = asyncio.create_task(search(user_adr))
         task2 = asyncio.create_task(report(user_adr))
         result_of_task1 = await task1
         result_of_task2 = await task2
        
         return result_of_task1, result_of_task2
    result_1, result_2 = await main(user_adr)
    return result_1, result_2

@app.route("/user_adr", methods=['GET'])
def user_adr():
    # 1. 웹으로부터 지갑주소 받기
    user_adr = request.args.get('user_adr', 'user_adr')
    print(user_adr)
    
    # 2. api에 지갑주소 넣어서 조회

    search_url = 'https://blockchain.info/rawaddr/'+user_adr+'?limit=10'
    report_url = 'https://www.bitcoinabuse.com/api/reports/check?address='+user_adr+'&api_token=GWGXyrT3Yil1PF8hZ49EEP6X8LiOkSr4KzA9bIDc'
    
    # 3. 결과데이터 리스트에 담기
    search_adrlist = requests.request("GET", search_url)
    report_adrlist = requests.request("GET", report_url)
    search_adr_json = search_adrlist.json()
    report_adr_json = report_adrlist.json()
    
    # 4. json타입으로 웹으로 전송
    return search_adr_json, report_adr_json
    

In [ ]:
user_adr = '1CRCqjGKgiNziUi9k5aqtbcLp3575BRQKB'

In [ ]:
def user_adr():
    # 1. 웹으로부터 지갑주소 받기
    user_adr = '1CRCqjGKgiNziUi9k5aqtbcLp3575BRQKB'    
    print(user_adr)
    
    # 2. api에 지갑주소 넣어서 조회

    search_url = 'https://blockchain.info/rawaddr/'+user_adr+'?limit=10'
    report_url = 'https://www.bitcoinabuse.com/api/reports/check?address='+user_adr+'&api_token=GWGXyrT3Yil1PF8hZ49EEP6X8LiOkSr4KzA9bIDc'
    
    # 3. 결과데이터 리스트에 담기
    search_adrlist = requests.request("GET", search_url)
    report_adrlist = requests.request("GET", report_url)
    search_adr_json = search_adrlist.json()
    report_adr_json = report_adrlist.json()
    
    # 4. json타입으로 웹으로 전송
    return (search_adr_json, report_adr_json)
    

In [ ]:
user_adr()